In [19]:
import pandas as pd
import glob
import os 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict


In [3]:
# TODO REFACTOR IN FUNCTIONS

In [12]:
def reader_assign_conf_pep(path, FDR_threshold):
    print(f'Reading {path}')
    df = pd.read_csv(path, sep = '\t')
    tot_peptides = len(df['unmodified sequence'].unique())
    print(f'With Shape: {df.shape[0]}')
    print(f'With unique peptides: {tot_peptides}')
    assert('delta_cn' in df.columns)
    df_filtered = df.loc[df['tdc q-value'] < FDR_threshold]
    print(f'Number of validated psm: {df_filtered.shape}')
    peptides = set(df_filtered['unmodified sequence'])
    val = len(peptides)
    val_rate = val / tot_peptides
    print(f'Number of validated unique peptides: {val}')
    print(f'Validation Rate: {np.round(val_rate * 100 , 2)} percent')
    return val, val_rate, peptides, df_filtered

In [77]:
def prepare_backticks(a, b):
    return  [f'({i}, {j})' for i, j in zip(a.values, b.values)]
def prepare_frontticks(a, b, c):
    return  [f'({i}, {j}, {k})' for i, j, k in zip(a.values, b.values, c.values)]


In [78]:
# Plot Helper
def plot_text(Y, T, position='top', color='black', font=None):
    font['color'] = color
    Y = np.array(Y)
    T = np.array(T)
    Y[np.where(Y > 0 )[0]]
    T[np.where(Y > 0 )[0]]
    change_val = [i for i in np.arange(len(Y) - 1) if Y[i] != Y[i - 1]]    
    weighted = [change_val[i] + (change_val[i+1] - change_val[i]) / 2 for i, x in enumerate(change_val[:-1])]
    X = [np.floor(change_val[i] + (change_val[i+1] - change_val[i]) / 2) for i, x in enumerate(change_val[:-1])]
    Y = Y[np.array(change_val[:-1])]
    T = T[np.array(change_val[:-1])]
    p_prev = 0 
    percent_diff = 20
    min_x = min(X)
    
    previous_plot = 0 
    for x, y, p in zip(X, Y, T):
        if position == 'bottom':
            delta = - (y/5.5)
        elif position == 'top': 
            delta = (y/12)
        if (p > p_prev + (p_prev/percent_diff)) or (p < p_prev - (p_prev/percent_diff)):
            previous_plot += 1
        if previous_plot == 2 or x == min_x: # delay the plotting
            if y != 0 : # Because log scale
                plt.text(x - 0.5 , y + delta , p, ha='left', **font)
                previous_plot = 0
                
        p_prev = p 

def plot_text_all(X, Y, T):
    for x, y, p in zip(X, Y, T):
        plt.text(x, y, p)
        #plt.text(x - 0.5 , y + (y/10), p)


In [58]:
# INPUTS 
basedir = '/cluster/work/grlab/projects/projects2020_OHSU/proteomics'
pipelines = ['OHSU', 'ETH']
samples = ['TCGA-C8-A12P-01A-11R-A115-07',
          'TCGA-AO-A0JM-01A-21R-A056-07',
          'TCGA-BH-A18V-01A-11R-A12D-07',
          'TCGA-A2-A0D2-01A-21R-A034-07',
          'TCGA-A2-A0SX-01A-12R-A084-07', 
          'TCGA-25-1319-01A-01R-1565-13',
          'TCGA-25-1313-01A-01R-1565-13',
          'TCGA-61-2008-01A-02R-1568-13',
          'TCGA-24-1431-01A-01R-1566-13',
          'TCGA-24-2298-01A-01R-1569-13']
#samples = samples[-2:-1]
print(samples)
FDR_limit = 0.05
MS_strategy = 'pool' #, 'joint', 'single' 
#pool: pool all experiments per pipeline
#joint: pool all experiments for both pipeline
#single: per experiment per pipeline

MS_FDR = 'crux'


['TCGA-C8-A12P-01A-11R-A115-07', 'TCGA-AO-A0JM-01A-21R-A056-07', 'TCGA-BH-A18V-01A-11R-A12D-07', 'TCGA-A2-A0D2-01A-21R-A034-07', 'TCGA-A2-A0SX-01A-12R-A084-07', 'TCGA-25-1319-01A-01R-1565-13', 'TCGA-25-1313-01A-01R-1565-13', 'TCGA-61-2008-01A-02R-1568-13', 'TCGA-24-1431-01A-01R-1566-13', 'TCGA-24-2298-01A-01R-1569-13']


In [80]:
sample_plot_dir = {}
run_type_plot_dir = {}    

# BRCA
samples = ['TCGA-C8-A12P-01A-11R-A115-07',
                  'TCGA-AO-A0JM-01A-21R-A056-07',
                  'TCGA-BH-A18V-01A-11R-A12D-07',
                  'TCGA-A2-A0D2-01A-21R-A034-07',
                  'TCGA-A2-A0SX-01A-12R-A084-07']
basedir = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Breast_1102'
filter_dir = os.path.join(basedir, 'filtering_samples', 'filters_19May_order_5ge_wAnnot_GPstar')

for sample in samples:
    sample_plot_dir[sample] = os.path.join(filter_dir, 'plots')
    run_type_plot_dir[sample] = 'BRCA'
    
# OV
samples = ['TCGA-25-1319-01A-01R-1565-13',
                  'TCGA-25-1313-01A-01R-1565-13',
                  'TCGA-61-2008-01A-02R-1568-13',
                  'TCGA-24-1431-01A-01R-1566-13',
                  'TCGA-24-2298-01A-01R-1569-13']
basedir = '/cluster/work/grlab/projects/projects2020_OHSU/peptides_generation/CANCER_eth/commit_c4dd02c_conf2_Frame_cap0_runs/TCGA_Ovarian_374'
filter_dir = os.path.join(basedir, 'filtering_samples', 'filters_19May_order_5ge_wAnnot_GPstar')

for sample in samples:
    sample_plot_dir[sample] = os.path.join(filter_dir, 'plots')
    run_type_plot_dir[sample] = 'BRCA'

# Parse: results for all experiments

In [68]:
samples_store_pep = {}
samples_store_rates = {}
experiment_pipelines_peptides =  defaultdict(dict)
experiments_pipelines_rates = defaultdict(dict)


for sample in samples:

    sample_short = '-'.join(sample.split('-')[0:3])
    samples_store_pep[sample] = experiment_pipelines_peptides
    samples_store_rates[sample] = experiments_pipelines_rates
    for pipeline in pipelines:
        path_single = os.path.join(basedir, pipeline, sample_short, 'assign_conf_per_experiment')
        path_pool_pipeline = os.path.join(basedir, pipeline, sample_short, 'assign_conf_pooled_FDR')
        path_pool_union = os.path.join(basedir, f'assign_conf_joint_to_{pipeline}', sample)
        
        path_test_OHSU = os.path.join(basedir, 'OHSU', sample_short, 'assign_conf_per_experiment')
        path_test_ETH = os.path.join(basedir, 'ETH', sample_short, 'assign_conf_per_experiment')
        experiment_list = [ i.split('/')[-1] for i in glob.glob(path_single + '/*')]

        for experiment in experiment_list:
            if pipeline == 'OHSU':
                original_name = experiment
                cut_name = experiment[1:]
            else:
                original_name = experiment
                cut_name = experiment
               
            if os.path.isfile(os.path.join(path_test_OHSU, 'J' + cut_name, 'assign-confidence.target.txt')) and \
                os.path.isfile(os.path.join(path_test_ETH, cut_name, 'assign-confidence.target.txt')): #Commun experiments
                
                
                # search 1 experiment, 1 pipeline  
                if MS_strategy == 'single':
                    df = os.path.join(path_single, original_name, 'assign-confidence.target.txt')
                # search all experiments, 1 pipeline
                if MS_strategy == 'pool':
                    df = os.path.join(path_pool_pipeline, f'tsearch-{original_name}.txt')
                # search all experiments, 1 union of pipelines
                if MS_strategy == 'joint':
                    df = os.path.join(path_pool_union, f'tsearch-{original_name}.txt')
                
                val, val_rate, peptides, _ = reader_assign_conf_pep(df, FDR_limit)
                samples_store_pep[sample][cut_name][pipeline] = peptides
                samples_store_rates[sample][cut_name][pipeline] = val_rate

                print('\n')


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-C8-A12P/assign_conf_pooled_FDR/tsearch-J0A53XGA.txt
With Shape: 1469
With unique peptides: 31
Number of validated psm: (0, 18)
Number of validated unique peptides: 0
Validation Rate: 0.0 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-C8-A12P/assign_conf_pooled_FDR/tsearch-J0A13AGA.txt
With Shape: 9545
With unique peptides: 258
Number of validated psm: (12, 18)
Number of validated unique peptides: 4
Validation Rate: 1.55 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-C8-A12P/assign_conf_pooled_FDR/tsearch-J0A51XGA.txt
With Shape: 389
With unique peptides: 11
Number of validated psm: (0, 18)
Number of validated unique peptides: 0
Validation Rate: 0.0 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-C8-A12P/assign_conf_pooled_FDR/tsearch-J0213XGA.txt
With Shape: 1511
With unique peptides: 38
Number of

With Shape: 42540
With unique peptides: 1134
Number of validated psm: (19, 18)
Number of validated unique peptides: 9
Validation Rate: 0.79 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-C8-A12P/assign_conf_pooled_FDR/tsearch-J02132GA.txt
With Shape: 1021
With unique peptides: 25
Number of validated psm: (9, 18)
Number of validated unique peptides: 1
Validation Rate: 4.0 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-C8-A12P/assign_conf_pooled_FDR/tsearch-J0211XGA.txt
With Shape: 712
With unique peptides: 14
Number of validated psm: (9, 18)
Number of validated unique peptides: 1
Validation Rate: 7.14 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-C8-A12P/assign_conf_pooled_FDR/tsearch-J0AN1XGA.txt
With Shape: 28548
With unique peptides: 757
Number of validated psm: (16, 18)
Number of validated unique peptides: 6
Validation Rate: 0.79 percent


Reading /cluster/work/grlab

With Shape: 108
With unique peptides: 4
Number of validated psm: (0, 18)
Number of validated unique peptides: 0
Validation Rate: 0.0 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-AO-A0JM/assign_conf_pooled_FDR/tsearch-J0A11XGA.txt
With Shape: 1194
With unique peptides: 20
Number of validated psm: (2, 18)
Number of validated unique peptides: 1
Validation Rate: 5.0 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-AO-A0JM/assign_conf_pooled_FDR/tsearch-J0253XGA.txt
With Shape: 228
With unique peptides: 9
Number of validated psm: (0, 18)
Number of validated unique peptides: 0
Validation Rate: 0.0 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-AO-A0JM/assign_conf_pooled_FDR/tsearch-J0AN3AGA.txt
With Shape: 25372
With unique peptides: 582
Number of validated psm: (149, 18)
Number of validated unique peptides: 9
Validation Rate: 1.55 percent


Reading /cluster/work/grlab/project

With Shape: 12154
With unique peptides: 418
Number of validated psm: (32, 18)
Number of validated unique peptides: 9
Validation Rate: 2.15 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-BH-A18V/assign_conf_pooled_FDR/tsearch-J0A532GA.txt
With Shape: 380
With unique peptides: 17
Number of validated psm: (0, 18)
Number of validated unique peptides: 0
Validation Rate: 0.0 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-BH-A18V/assign_conf_pooled_FDR/tsearch-J0A11XGA.txt
With Shape: 1517
With unique peptides: 55
Number of validated psm: (0, 18)
Number of validated unique peptides: 0
Validation Rate: 0.0 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-BH-A18V/assign_conf_pooled_FDR/tsearch-J0253XGA.txt
With Shape: 162
With unique peptides: 7
Number of validated psm: (0, 18)
Number of validated unique peptides: 0
Validation Rate: 0.0 percent


Reading /cluster/work/grlab/project

With Shape: 8452
With unique peptides: 228
Number of validated psm: (19, 18)
Number of validated unique peptides: 9
Validation Rate: 3.95 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-A2-A0D2/assign_conf_pooled_FDR/tsearch-J0213XGA.txt
With Shape: 1407
With unique peptides: 37
Number of validated psm: (0, 18)
Number of validated unique peptides: 0
Validation Rate: 0.0 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-A2-A0D2/assign_conf_pooled_FDR/tsearch-J0AN01GA.txt
With Shape: 14830
With unique peptides: 416
Number of validated psm: (11, 18)
Number of validated unique peptides: 5
Validation Rate: 1.2 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-A2-A0D2/assign_conf_pooled_FDR/tsearch-J0A532GA.txt
With Shape: 208
With unique peptides: 10
Number of validated psm: (0, 18)
Number of validated unique peptides: 0
Validation Rate: 0.0 percent


Reading /cluster/work/grlab/pro

...

With Shape: 27011
With unique peptides: 1607
Number of validated psm: (385, 18)
Number of validated unique peptides: 41
Validation Rate: 2.55 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/ETH/TCGA-61-2008/assign_conf_pooled_FDR/tsearch-0AN3XGA.txt
With Shape: 106887
With unique peptides: 6373
Number of validated psm: (1097, 18)
Number of validated unique peptides: 160
Validation Rate: 2.51 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/ETH/TCGA-61-2008/assign_conf_pooled_FDR/tsearch-0A532GA.txt
With Shape: 70300
With unique peptides: 4172
Number of validated psm: (894, 18)
Number of validated unique peptides: 107
Validation Rate: 2.56 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/ETH/TCGA-61-2008/assign_conf_pooled_FDR/tsearch-0A112GA.txt
With Shape: 70859
With unique peptides: 4251
Number of validated psm: (723, 18)
Number of validated unique peptides: 101
Validation Rate: 2.38 percent


Reading /

With Shape: 62690
With unique peptides: 10553
Number of validated psm: (1145, 18)
Number of validated unique peptides: 353
Validation Rate: 3.35 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-24-1431/assign_conf_pooled_FDR/tsearch-J0253AGA.txt
With Shape: 13106
With unique peptides: 2142
Number of validated psm: (308, 18)
Number of validated unique peptides: 93
Validation Rate: 4.34 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-24-1431/assign_conf_pooled_FDR/tsearch-J0251AGA.txt
With Shape: 7865
With unique peptides: 1283
Number of validated psm: (195, 18)
Number of validated unique peptides: 56
Validation Rate: 4.36 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-24-1431/assign_conf_pooled_FDR/tsearch-J02501GA.txt
With Shape: 7472
With unique peptides: 1220
Number of validated psm: (194, 18)
Number of validated unique peptides: 55
Validation Rate: 4.51 percent


Reading

With Shape: 34245
With unique peptides: 1477
Number of validated psm: (158, 18)
Number of validated unique peptides: 44
Validation Rate: 2.98 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/ETH/TCGA-24-1431/assign_conf_pooled_FDR/tsearch-0253XGA.txt
With Shape: 33091
With unique peptides: 1431
Number of validated psm: (152, 18)
Number of validated unique peptides: 42
Validation Rate: 2.94 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/ETH/TCGA-24-1431/assign_conf_pooled_FDR/tsearch-0A132GA.txt
With Shape: 53206
With unique peptides: 2300
Number of validated psm: (231, 18)
Number of validated unique peptides: 53
Validation Rate: 2.3 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/ETH/TCGA-24-1431/assign_conf_pooled_FDR/tsearch-0213AGA.txt
With Shape: 57430
With unique peptides: 2506
Number of validated psm: (246, 18)
Number of validated unique peptides: 64
Validation Rate: 2.55 percent


Reading /cluste

With Shape: 45035
With unique peptides: 14752
Number of validated psm: (1308, 18)
Number of validated unique peptides: 517
Validation Rate: 3.5 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-24-2298/assign_conf_pooled_FDR/tsearch-J0A501GA.txt
With Shape: 22263
With unique peptides: 7186
Number of validated psm: (794, 18)
Number of validated unique peptides: 302
Validation Rate: 4.2 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-24-2298/assign_conf_pooled_FDR/tsearch-J0211AGA.txt
With Shape: 15255
With unique peptides: 4932
Number of validated psm: (521, 18)
Number of validated unique peptides: 183
Validation Rate: 3.71 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/OHSU/TCGA-24-2298/assign_conf_pooled_FDR/tsearch-J02512GA.txt
With Shape: 6029
With unique peptides: 1932
Number of validated psm: (238, 18)
Number of validated unique peptides: 78
Validation Rate: 4.04 percent


Readin

With Shape: 58918
With unique peptides: 4304
Number of validated psm: (336, 18)
Number of validated unique peptides: 104
Validation Rate: 2.42 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/ETH/TCGA-24-2298/assign_conf_pooled_FDR/tsearch-0AN01GA.txt
With Shape: 62311
With unique peptides: 4559
Number of validated psm: (355, 18)
Number of validated unique peptides: 109
Validation Rate: 2.39 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/ETH/TCGA-24-2298/assign_conf_pooled_FDR/tsearch-0AN1XGA.txt
With Shape: 63913
With unique peptides: 4679
Number of validated psm: (359, 18)
Number of validated unique peptides: 111
Validation Rate: 2.37 percent


Reading /cluster/work/grlab/projects/projects2020_OHSU/proteomics/ETH/TCGA-24-2298/assign_conf_pooled_FDR/tsearch-0AN32GA.txt
With Shape: 73572
With unique peptides: 5417
Number of validated psm: (454, 18)
Number of validated unique peptides: 130
Validation Rate: 2.4 percent


Reading /cl

In [69]:
compare = {'sample' : [], 
          'filter_' : [], 
          'pep_size_ohsu' : [], 
          'pep_size_eth' : [], 
          'pep_size_intersection' : [], 
          'pep_size_ohsu\eth' : [], 
          'pep_size_eth\ohsu' : []}

for sample, experiments_ in samples_store_pep.items():
    for experiment, pipelines_ in experiments_.items():
        compare['sample'].append(sample)
        compare['filter_'].append(experiment)
        compare['pep_size_ohsu'].append(len(pipelines_['OHSU']))
        compare['pep_size_eth'].append(len(pipelines_['ETH']))
        compare['pep_size_ohsu\eth'].append(len(pipelines_['OHSU'].difference(pipelines_['ETH'])))
        compare['pep_size_eth\ohsu'].append(len(pipelines_['ETH'].difference(pipelines_['OHSU'])))
        compare['pep_size_intersection'].append(len(pipelines_['ETH'].intersection(pipelines_['ETH'])))

# Plot: Results per experiment

In [70]:
df = pd.DataFrame(compare)

In [71]:
display(df.head())
print(df.shape)

,sample,filter_,pep_size_ohsu,pep_size_eth,pep_size_intersection,pep_size_ohsu\eth,pep_size_eth\ohsu
0,TCGA-C8-A12P-01A-11R-A115-07,0A53XGA,450,127,127,351,28
1,TCGA-C8-A12P-01A-11R-A115-07,0A13AGA,723,152,152,592,21
2,TCGA-C8-A12P-01A-11R-A115-07,0A51XGA,315,86,86,247,18
3,TCGA-C8-A12P-01A-11R-A115-07,0213XGA,284,118,118,208,42
4,TCGA-C8-A12P-01A-11R-A115-07,0AN01GA,753,109,109,660,16


(350, 7)


In [52]:
sample = 'TCGA-25-1319-01A-01R-1565-13'

In [53]:
order_file = os.path.join(sample_plot_dir[sample], f'{sample}_Barplot_sorting.tsv.gz')
order_file = pd.read_csv(order_file, sep = '\t')
display(order_file.head())
print(order_file.shape)

,Unnamed: 0,sample,filter_,size_ohsu,size_eth,size_intersection,size_ohsu\eth,size_eth\ohsu,filter_foreground_target,filter_foreground_reads,filter_foreground_samples,filter_background_reads,filter_background_samples,cohort,motif,index,filter_background,filter_foreground
0,7,TCGA-25-1319-01A-01R-1565-13,02501GA,23972,20052,8874,15098,11178,0,2,5,0,1,G,A,0,"(0, 1)","(0, 2, 5)"
1,30,TCGA-25-1319-01A-01R-1565-13,02512GA,24766,20638,9166,15600,11472,0,2,5,1,2,G,A,1,"(1, 2)","(0, 2, 5)"
2,14,TCGA-25-1319-01A-01R-1565-13,0251AGA,24913,20683,9193,15720,11490,0,2,5,1,A,G,A,2,"(1, A)","(0, 2, 5)"
3,8,TCGA-25-1319-01A-01R-1565-13,0251XGA,24913,20683,9193,15720,11490,0,2,5,1,X,G,A,3,"(1, X)","(0, 2, 5)"
4,15,TCGA-25-1319-01A-01R-1565-13,02532GA,29573,24940,11086,18487,13854,0,2,5,3,2,G,A,4,"(3, 2)","(0, 2, 5)"


(35, 18)


In [ ]:
df.loc[df['sample'] == sample]

In [73]:
df_plot = order_file.merge(df, on = ['sample', 'filter_'], how = 'inner')
print(df_plot.shape)
df_plot = df_plot.sort_values('index')

(35, 23)


In [74]:
df_plot.head()

,Unnamed: 0,sample,filter_,size_ohsu,size_eth,size_intersection,size_ohsu\eth,size_eth\ohsu,filter_foreground_target,filter_foreground_reads,...,cohort,motif,index,filter_background,filter_foreground,pep_size_ohsu,pep_size_eth,pep_size_intersection,pep_size_ohsu\eth,pep_size_eth\ohsu
0,7,TCGA-25-1319-01A-01R-1565-13,02501GA,23972,20052,8874,15098,11178,0,2,...,G,A,0,"(0, 1)","(0, 2, 5)",76,37,37,49,10
1,30,TCGA-25-1319-01A-01R-1565-13,02512GA,24766,20638,9166,15600,11472,0,2,...,G,A,1,"(1, 2)","(0, 2, 5)",78,38,38,51,11
2,14,TCGA-25-1319-01A-01R-1565-13,0251AGA,24913,20683,9193,15720,11490,0,2,...,G,A,2,"(1, A)","(0, 2, 5)",78,38,38,51,11
3,8,TCGA-25-1319-01A-01R-1565-13,0251XGA,24913,20683,9193,15720,11490,0,2,...,G,A,3,"(1, X)","(0, 2, 5)",78,38,38,51,11
4,15,TCGA-25-1319-01A-01R-1565-13,02532GA,29573,24940,11086,18487,13854,0,2,...,G,A,4,"(3, 2)","(0, 2, 5)",96,51,51,59,14


In [85]:
run_type = run_type_plot_dir[sample]
df = df_plot 
back_ticks = df['filter_background'] 
front_ticks = df['filter_foreground']
number of peptides

In [ ]:
ticks_fontsize = 12
axislabels_fontsize = 12
legend_fontsize = 12
axes_fontsize = 15
serie_index = df_plot['index']
serie_intersection = df_plot['pep_size_intersection']
serie_eth = df_plot['pep_size_eth']
serie_ohsu = df_plot['pep_size_ohsu']
y_label = 

In [87]:
def plot_intersection_bars(ticks_fontsize, axislabels_fontsize, 
                          legend_fontsize, axes_fontsize, run_type, 
                           serie_index, serie_intersection, serie_eth, serie_ohsu
                          y_label)
#TODO FUNCTION!!
# Plot
text_font = {'size':'12', 'weight':'normal'}

xaxis_label = 'GTEX (reads, samples)'
yaxis_label = f'{run_type.upper()} (min, reads, samples)'


colorgrid = 'grey'
alpha_grid = 0.3
marker_size = 10
mew = 4
color1 = 'gold'
color2 = 'crimson'
color3 = 'mediumblue'
color4 = 'tomato'
colorgrid = 'grey'


fig, ax1 = plt.subplots(figsize=(15, 6))
ax2 =  ax1.secondary_xaxis('top')   
#plt.figure(figsize=(15, 6), frameon=False, dpi=100)
plt.grid(b=True, axis = 'both', which='major', color=colorgrid, linestyle='-', alpha=alpha_grid)
plt.grid(b=False, axis = 'both', which='minor', color=colorgrid, linestyle='--', alpha=alpha_grid)
width = 0.4
plt.bar(serie_index, serie_intersection, width=width, 
        color=color1, label='Intersection size')
plt.plot(serie_index, serie_eth, alpha=1, color=color3,
         linestyle = 'None', markerfacecolor='None', marker="_", markersize=marker_size, markeredgewidth=mew,
         label = 'Total set size Graph Pipeline')
plt.plot(serie_index, serie_ohsu, alpha=1, color=color2,
         linestyle = 'None', markerfacecolor='None', marker="_", markersize=marker_size, markeredgewidth=mew,
         label = 'Total set size Junction Pipeline')

plot_text(serie_ohsu, serie_ohsu, 'top', color=color2, font=text_font)
plot_text(serie_eth, serie_eth, 'top', color=color3, font=text_font)
plot_text(serie_intersection, serie_intersection, color=color4, font=text_font)

plt.yscale('log')
max_scale = np.max([serie_ohsu.values, serie_eth.values])



ax1.set_xticks(np.arange(len(serie_index)), 
           labels = front_ticks,
           rotation = 90, 
           ha = 'center', 
           fontsize=ticks_fontsize)

ax2.set_xticks(np.arange(len(serie_index)), 
           labels = back_ticks,
           rotation = 90, 
           ha = 'center', 
           fontsize=ticks_fontsize)


plt.legend(fontsize=legend_fontsize)
plt.ylabel(fontsize=axes_fontsize)
ax2.set_xlabel(xaxis_label, fontsize=axes_fontsize)
ax1.set_xlabel(yaxis_label, fontsize=axes_fontsize)



# save = False
# name = 'Barplot_ovelap'
# save_path = os.path.join(plot_dir, f'{base_plot}_{name}.pdf')
# if save:
#     print("save to {}".format(save_path))
#     plt.savefig(save_path, bbox_inches='tight')
    
plt.show()

SyntaxError: invalid syntax (1809885965.py, line 3)